In [43]:
%matplotlib inline
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from init_objects import *
from functions.helpers import organise_data
from qe_model import *
from matplotlib import style
from SALib.sample import latin
import scipy.stats as stats
from hurst import compute_Hc, random_walk
import math
import scipy.stats as st
import datetime
import pandas_datareader.data as web
#from functions.find_bubbles import *
import matplotlib.transforms as mtransforms

In [35]:
def get_specific_bond_moments(full_series, bond_number):
    """Get a vector with the moments of a specific bootstrap"""
    return np.array([full_series[i][bond_number] for i in range(len(full_series))])

# Estimating uncertain parameters

## Collect data

In [2]:
bond_index_tickers = ["BAMLCC0A0CMTRIV", "BAMLCC0A1AAATRIV", "BAMLCC0A2AATRIV", "BAMLCC0A3ATRIV", 
                     "BAMLCC0A4BBBTRIV", "BAMLCC1A013YTRIV", "BAMLCC2A035YTRIV", "BAMLCC3A057YTRIV",
                     "BAMLCC4A0710YTRIV", "BAMLCC7A01015YTRIV", "BAMLCC8A015PYTRIV", "BAMLEM1BRRAAA2ACRPITRIV", 
                      "BAMLEM1RAAA2ALCRPIUSTRIV", 
                      "BAMLHE00EHYITRIV", "BAMLHYH0A0HYM2TRIV", "BAMLHYH0A1BBTRIV","BAMLHYH0A2BTRIV", "BAMLHYH0A3CMTRIV"]

In [3]:
start = datetime.datetime(2010, 4, 17)
end = datetime.datetime(2018, 1, 31)

bondindices = web.DataReader(bond_index_tickers, "fred", start, end)

autocors_emp = pd.DataFrame()
for col in bondindices:
    autocors_emp[col] = [bondindices[col].pct_change().autocorr(lag=lag) for lag in range(25)]

## Determine initial / calibrated parameters

In [4]:
params_nb = {"fundamental_value": 166, 
             "trader_sample_size": 22, 
             "n_traders": 1000,
             "ticks": 600, 
             "std_fundamental": 0.053,
             "init_assets": 740,
             'spread_max': 0.004,
             'money_multiplier': 2.2,
             "horizon": 200,
             "std_noise": 0.049, 
             "w_random": 0.08,
             "strat_share_chartists": 0.08,
             "base_risk_aversion": 1.051,
             "fundamentalist_horizon_multiplier": 3.8,
             "trades_per_tick": 1, "mutation_intensity": 0.0477,
             "average_learning_ability": 0.05, 
             "bond_mean_reversion": 0.0, 'cb_pf_range': 0.05,
             "qe_perc_size": 0.16, "cb_size": 0.02, "qe_asset_index": 0, "qe_start": 2, "qe_end":598}

## Calculate empirical moments

In [15]:
# calculate hursts
hursts = []
for col in bondindices:
    H, c, data = compute_Hc(bondindices[col].dropna(), kind='price', simplified=True)
    hursts.append(H)

In [18]:
av_autocor = np.mean(autocors_emp.mean())
av_autocor_abs = np.mean(autocors_emp.abs().mean())
av_kurtosis = np.mean(bondindices.pct_change().kurtosis())
av_hurst = np.mean(hursts)

In [19]:
emp_moments = np.array([
    av_autocor,
    av_autocor_abs,
    av_kurtosis,
    av_hurst
    ])
emp_moments

array([0.05034916, 0.06925489, 4.16055312, 0.71581425])

To account for the fact that some of these moments might correlate over different Monte Carlo Simulations, the MSM seeks to obtain a variance covariance matrix of the moments. Since there are multiple bond indices, I use these to create a covariance matrix of empirical moments.

In [32]:
bond_autocors = list(autocors_emp.mean())
bond_autocors_abs = list(autocors_emp.abs().mean())
bond_kurts = list(bondindices.pct_change().kurtosis())

In [34]:
all_bond_moments = [bond_autocors, bond_autocors_abs, bond_kurts, hursts]

In [33]:
emp_moments

array([0.05034916, 0.06925489, 4.16055312, 0.71581425])

In [36]:
moments_b = [get_specific_bond_moments(all_bond_moments, n) for n in range(len(hursts))]

In [38]:
W_hat = 1.0 / len(hursts) * sum([np.dot(np.array([(mb - emp_moments)]).transpose(), np.array([(mb - emp_moments)])) for mb in moments_b])
W = np.linalg.inv(W_hat)

In [49]:
np.save('distr_weighting_matrix', W)

To start the estimation procedure, I first sample the parameter space using Latin Hypercube sampling

In [52]:
population_size = 3

problem = {
  'num_vars': 7,
  'names': ['std_noise', "w_random", "strat_share_chartists", 
            "base_risk_aversion", "fundamentalist_horizon_multiplier",
            "mutation_intensity", "average_learning_ability"],
  'bounds': [[0.03, 0.09], [0.02, 0.15], [0.02, 0.3],
            [0.05, 3.0], [1.0, 5.0], 
            [0.05,0.5], [0.01, 0.8]]
}

In [53]:
latin_hyper_cube = latin.sample(problem=problem, N=population_size)
latin_hyper_cube = latin_hyper_cube.tolist()
with open('hypercube.txt', 'w') as f:
    json.dump(latin_hyper_cube, f)
initial_params = latin_hyper_cube[0]
initial_params

[0.05789905848770526,
 0.05312093723012003,
 0.2822482076893787,
 2.2460127763420896,
 4.664215077300264,
 0.17921674504763818,
 0.3548645096002974]

## Estimate in py file

In [55]:
with open('estimated_params.json', 'r') as f:
    est_params = json.loads(f.read())

In [57]:
for i, name in enumerate(problem['names']):
    params_nb[name] = est_params[i]

In [58]:
params_nb

{'fundamental_value': 166,
 'trader_sample_size': 22,
 'n_traders': 1000,
 'ticks': 600,
 'std_fundamental': 0.053,
 'init_assets': 740,
 'spread_max': 0.004,
 'money_multiplier': 2.2,
 'horizon': 200,
 'std_noise': 0.07278259263439237,
 'w_random': 0.14923007828230175,
 'strat_share_chartists': 0.2312452732431615,
 'base_risk_aversion': 1.1684997364091114,
 'fundamentalist_horizon_multiplier': 1.6608735823712282,
 'trades_per_tick': 1,
 'mutation_intensity': 0.3082243423666834,
 'average_learning_ability': 0.28994144339194206,
 'bond_mean_reversion': 0.0,
 'cb_pf_range': 0.05,
 'qe_perc_size': 0.16,
 'cb_size': 0.02,
 'qe_asset_index': 0,
 'qe_start': 2,
 'qe_end': 598}

In [20]:
# col_autocors = []
# col_autocors_abs = []
# col_kurts = []
# col_hursts = hursts

# for col in bondindices:
#     rets_mean.append(pd.Series(rets).mean())
#     #rets_std.append(pd.Series(rets).std())
#     rets_autocor.append(autocorrelation_returns(rets, 25))